In [12]:
! py -m pip install mlxtend --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.4.1 requires packaging<22.0,>=0.21, but you have packaging 23.2 which is incompatible.



  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 1.4/1.4 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 11.1/11.1 MB 18.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.1.7
    Uninstalling mlxtend-0.1.7:
      Successfully uninstalled mlxtend-0.1.7


In [81]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI
from prompts import *

In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

# Setup
fake = Faker()
np.random.seed(42)
random.seed(42)

# -----------------------
# 1. Generate Customers
# -----------------------
num_customers = 100
customer_data = []

for i in range(1, num_customers + 1):
    is_male = random.choice([True, False])
    first_name = fake.first_name_male() if is_male else fake.first_name_female()
    last_name = fake.last_name()
    gender = 'M' if is_male else 'F'
    dob = fake.date_of_birth(minimum_age=18, maximum_age=75)
    age = (datetime.now().date() - dob).days // 365
    income = random.randint(20000, 150000)
    credit_score = int(np.clip(np.random.normal(loc=600 + (income / 1000), scale=50), 300, 850))

    customer_data.append({
        "Customer_ID": f"CUST{i:04d}",
        "First_Name": first_name,
        "Last_Name": last_name,
        "Gender": gender,
        "Date_of_Birth": dob,
        "Age": age,
        "Email": fake.email(),
        "Phone": fake.phone_number(),
        "Address": fake.address().replace("\n", ", "),
        "City": fake.city(),
        "State": fake.state(),
        "Postal_Code": fake.postcode(),
        "Country": "USA",
        "Annual_Income": income,
        "Credit_Score": credit_score
    })

df_customers = pd.DataFrame(customer_data)
df_customers.to_csv("customers.csv", index=False)

# ----------------------------
# 2. Generate Transactions
# ----------------------------
transaction_data = []
categories = ['Grocery', 'Fuel', 'Dining', 'Travel', 'Medical', 'Baby', 'Education', 'Entertainment', 'Home', 'Salary']
merchants = {
    'Grocery': ['Walmart', 'Whole Foods', 'Kroger'],
    'Fuel': ['Shell', 'Exxon', 'Chevron'],
    'Dining': ['McDonald\'s', 'Starbucks', 'Chipotle'],
    'Travel': ['Delta', 'Airbnb', 'Uber'],
    'Medical': ['CVS', 'Walgreens', 'Urgent Care'],
    'Baby': ['BabyGap', 'Target', 'Pampers Store'],
    'Education': ['Coursera', 'Udemy', 'University Bookstore'],
    'Entertainment': ['Netflix', 'AMC Theatres', 'Spotify'],
    'Home': ['Home Depot', 'Lowe\'s', 'IKEA'],
    'Salary': ['Company Payroll', 'Direct Deposit']
}

txn_id = 1
for customer in df_customers['Customer_ID']:
    num_txns = random.randint(5, 15)
    for _ in range(num_txns):
        category = random.choice(categories)
        merchant = random.choice(merchants[category])
        amount = round(random.uniform(10, 2000) if category != 'Salary' else random.uniform(3000, 10000), 2)
        date = datetime.now() - timedelta(days=random.randint(0, 28))
        
        transaction_data.append({
            "Transaction_ID": f"TXN{txn_id:06d}",
            "Customer_ID": customer,
            "Timestamp": date.strftime("%Y-%m-%d %H:%M:%S"),
            "Merchant": merchant,
            "Category": category,
            "Amount": amount,
            "Description": f"{category} - {merchant}"
        })
        txn_id += 1

df_transactions = pd.DataFrame(transaction_data)
df_transactions.to_csv("transactions.csv", index=False)

# -------------------------
# 3. Generate Product Table
# -------------------------



TypeError: Random.randint() takes 3 positional arguments but 4 were given

In [3]:
import pandas as pd

# Load CSVs
df_customers = pd.read_csv("customers.csv")
df_transactions = pd.read_csv("transactions.csv")

# Ensure timestamp is in datetime format
df_transactions["Timestamp"] = pd.to_datetime(df_transactions["Timestamp"])

# Filter only last 4 weeks of transactions
latest_date = df_transactions["Timestamp"].max()
cutoff_date = latest_date - pd.Timedelta(days=28)
df_recent = df_transactions[df_transactions["Timestamp"] >= cutoff_date]

# Aggregate features per customer
features = df_recent.groupby("Customer_ID").agg(
    Total_Spend=("Amount", "sum"),
    Num_Transactions=("Transaction_ID", "count"),
    Avg_Txn_Amount=("Amount", "mean"),
    Max_Txn_Amount=("Amount", "max"),
    Has_Salary_Credit=("Category", lambda x: int("Salary" in x.values))
).reset_index()

# Pivot category spend into separate columns
category_spend = df_recent.pivot_table(
    index="Customer_ID",
    columns="Category",
    values="Amount",
    aggfunc="sum",
    fill_value=0
).add_prefix("Spend_").reset_index()

# Merge with customer demographics
df_feature_store = (
    features
    .merge(category_spend, on="Customer_ID", how="left")
    .merge(df_customers[["Customer_ID", "Age", "Annual_Income", "Credit_Score"]], on="Customer_ID", how="left")
)

# Save to CSV
df_feature_store.to_csv("feature_store.csv", index=False)


In [77]:
import pandas as pd

# Full product list with detailed columns
products = [
    ("P001", "Starter Credit Card", "Credit Card", "Low",
     "1% cashback on all purchases",
     "New credit users, low spenders",
     "Income > 20,000 and Credit Score > 600",
     "$25 cashback on first $300 spend"),
    
    ("P002", "Everyday Saver", "Savings Account", "Low",
     "4% interest, no maintenance fees",
     "Customers starting savings",
     "Open to all with minimum $100 deposit",
     "$50 bonus for maintaining $1,000 balance for 3 months"),
    
    ("P003", "Smart Shopper Card", "Credit Card", "Mid",
     "2% cashback on groceries & fuel",
     "High grocery and fuel spenders",
     "Income > 30,000 and Credit Score > 650",
     "$50 cashback on first $500 spend"),
    
    ("P004", "Smart Budget Account", "Savings + Budget", "Mid",
     "5% APY on goal-based savings",
     "Budget-conscious savers",
     "Income > 25,000",
     "Free premium budgeting tools for 6 months"),
    
    ("P005", "Travel Rewards Elite", "Credit Card", "High",
     "3x travel points, lounge access",
     "Frequent travelers",
     "Income > 70,000 and Credit Score > 700",
     "$200 travel voucher after $5,000 spend"),
    
    ("P006", "Elite Wealth Plan", "Investment", "High",
     "6-8% ROI, personal wealth advisor",
     "High-income professionals",
     "Income > 100,000 and Age > 30",
     "1% bonus returns for first year"),
    
    ("P007", "Family Future Plan", "Insurance/Savings", "Mid-High",
     "Child savings + life insurance combo",
     "Families, new parents",
     "Married, Age 25-45",
     "$100 bonus on first year premium"),
    
    ("P008", "Student Flex Account", "Savings", "Student",
     "No fees, education budgeting tools",
     "Students under 25",
     "Age < 25",
     "$100 top-up on monthly deposits over $200"),
    
    ("P009", "Home Advantage Loan", "Loan", "Mid-High",
     "Low-interest home loans",
     "First-time home buyers",
     "Home-related spend > $10,000",
     "Reduced interest rate for bundled insurance"),
    
    ("P010", "Retirement Essentials", "Wealth + Health", "Senior",
     "Pension fund + health perks",
     "Customers over 60",
     "Age > 60",
     "$500 wellness bonus for first year"),
    
    ("P011", "FlexFuel Card", "Credit Card", "Low-Mid",
     "3% cashback on fuel",
     "Regular commuters",
     "Fuel spend > $150/month",
     "$50 fuel voucher after $500 spend"),
    
    ("P012", "Digital Nomad Saver", "Savings Account", "Mid",
     "1.5% bonus APY for international use",
     "Frequent travelers, remote workers",
     "Travel spend > $500/month",
     "No international transfer fees for 6 months"),
    
    ("P013", "HealthSecure Plan", "Health + Savings", "Mid",
     "HSA integration, telehealth access",
     "Health-conscious individuals",
     "Medical spend > $200/month",
     "$200 credit toward medical expenses"),
    
    ("P014", "GreenLife Investment", "ESG Investment", "Mid-High",
     "5-9% returns in sustainable funds",
     "Eco-conscious investors",
     "Interest in ESG, Income > 40,000",
     "$100 green bonus + 0.5% bonus returns"),
    
    ("P015", "Weekend Explorer Card", "Credit Card", "Mid",
     "2.5% cashback on dining & entertainment",
     "Social and active lifestyle",
     "Dining + entertainment spend > $400/month",
     "Free concert ticket after $2,000 spend"),
    
    ("P016", "CashBuilder Certificate", "CD / Fixed Deposit", "Low-Mid",
     "4.75% fixed interest (6 months)",
     "Idle balance savers",
     "Idle funds > $5,000",
     "$25 bonus for auto-renewal"),
    
    ("P017", "Lifestyle Bundle Plus", "Bundle (3-in-1)", "High",
     "Credit + Wealth + Travel perks",
     "Affluent, multi-product users",
     "Income > 100,000",
     "$300 statement credit + concierge onboarding"),
    
    ("P018", "BabyStart Trust Plan", "Child Investment", "Mid",
     "Custodial account for education",
     "Parents with young children",
     "Children < 5 years",
     "1st year fees waived + $100 education bonus"),
    
    ("P019", "MoveSmart Relocation Loan", "Loan", "Mid",
     "0% for 6 months, flexible repayment",
     "Customers with large recent spends",
     "One-time spend > $5,000",
     "$150 moving voucher"),
    
    ("P020", "SideHustle Account", "Business Checking", "Low-Mid",
     "No fees, invoice management",
     "Freelancers, side businesses",
     "Self-employed, 3+ biz txns/month",
     "$75 bonus for linking payment gateway")
]

# Create DataFrame
columns = ["Product_ID", "Product_Name", "Product_Type", "Tier",
           "Features_Benefits", "Target_Behavior", "Eligibility_Criteria", "Special_Offer"]

df_products = pd.DataFrame(products, columns=columns)

for col in ["Features_Benefits", "Special_Offer"]:
    df_products[col] = df_products[col].str.replace("$", "USD ", regex=False)

# Save to CSV
df_products.to_csv("products.csv", index=False)

print("✅ products.csv created with detailed product information.")


✅ products.csv created with detailed product information.


In [37]:
import pandas as pd

# Load data
df_transactions = pd.read_csv("transactions.csv")
df_customers = pd.read_csv("customers.csv")
df_feature_store = pd.read_csv("feature_store.csv")

# Ensure Timestamp is datetime
df_transactions["Timestamp"] = pd.to_datetime(df_transactions["Timestamp"])

# 1️⃣ Calculate Aggregation_Days per customer
agg_days = df_transactions.groupby("Customer_ID").agg(
    First_Txn_Date=("Timestamp", "min"),
    Last_Txn_Date=("Timestamp", "max")
)
agg_days["Aggregation_Days"] = (agg_days["Last_Txn_Date"] - agg_days["First_Txn_Date"]).dt.days + 1
agg_days = agg_days[["Aggregation_Days"]].reset_index()

# 2️⃣ Calculate Spend Variability (std deviation of Amount)
spend_var = df_transactions.groupby("Customer_ID").agg(
    Spend_Variability=("Amount", "std")
).fillna(0).reset_index()

# 3️⃣ Calculate Salary_to_Spend_Ratio
salary_spend = df_transactions.groupby(["Customer_ID", "Category"]).agg(Total_Category_Spend=("Amount", "sum")).reset_index()
salary = salary_spend[salary_spend["Category"] == "Salary"][["Customer_ID", "Total_Category_Spend"]].rename(columns={"Total_Category_Spend": "Total_Salary"})
total_spend = df_feature_store[["Customer_ID", "Total_Spend"]]

salary_ratio = pd.merge(total_spend, salary, on="Customer_ID", how="left").fillna(0)
salary_ratio["Salary_to_Spend_Ratio"] = salary_ratio["Total_Salary"] / salary_ratio["Total_Spend"]
salary_ratio = salary_ratio[["Customer_ID", "Salary_to_Spend_Ratio"]]

# 4️⃣ Top Spend Category
top_category = df_transactions[df_transactions["Category"] != "Salary"].groupby(["Customer_ID", "Category"]).agg(
    Category_Spend=("Amount", "sum")
).reset_index()

top_spend_cat = top_category.loc[top_category.groupby("Customer_ID")["Category_Spend"].idxmax()]
top_spend_cat = top_spend_cat[["Customer_ID", "Category"]].rename(columns={"Category": "Top_Spend_Category"})

# 5️⃣ Idle Balance Estimate
income = df_customers[["Customer_ID", "Annual_Income"]]
idle_balance = pd.merge(total_spend, income, on="Customer_ID", how="left")
idle_balance["Idle_Balance_Estimate"] = idle_balance["Annual_Income"] - idle_balance["Total_Spend"]
idle_balance = idle_balance[["Customer_ID", "Idle_Balance_Estimate"]]

# Merge all new features
df_enhanced = df_feature_store.merge(agg_days, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(spend_var, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(salary_ratio, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(top_spend_cat, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(idle_balance, on="Customer_ID", how="left")

# Fill any remaining NaNs
df_enhanced = df_enhanced.fillna({
    "Spend_Variability": 0,
    "Salary_to_Spend_Ratio": 0,
    "Top_Spend_Category": "Unknown",
    "Aggregation_Days": 28  # Default if only 1 txn
})

# Save enhanced feature store
df_enhanced.to_csv("feature_store_enhanced.csv", index=False)

print("✅ Enhanced feature_store_enhanced.csv created.")


✅ Enhanced feature_store_enhanced.csv created.


In [79]:
import pandas as pd
import random

# Load customer data and product catalog
df_customers = pd.read_csv("customers.csv")
df_products = pd.read_csv("products.csv")

# Prepare list to store ownership records
customer_products = []

# Define logic-based exclusions
def get_eligible_products(customer_row):
    age = customer_row['Age']
    eligible_products = df_products.copy()

    # Exclude Student Account for non-students
    if age > 25:
        eligible_products = eligible_products[eligible_products['Product_ID'] != 'P008']
    
    # Exclude Retirement plan for young customers
    if age < 55:
        eligible_products = eligible_products[eligible_products['Product_ID'] != 'P010']
    
    return eligible_products['Product_ID'].tolist()

# Generate ownership for each customer
for idx, customer in df_customers.iterrows():
    cust_id = customer['Customer_ID']
    
    # Get eligible products
    eligible_products = get_eligible_products(customer)
    
    # Randomly assign 1 to 3 products
    owned_count = random.randint(1, 3)
    owned_products = random.sample(eligible_products, owned_count)
    
    for prod_id in owned_products:
        customer_products.append({
            "Customer_ID": cust_id,
            "Product_ID": prod_id
        })

# Create DataFrame
df_customer_products = pd.DataFrame(customer_products)

# Save to CSV
df_customer_products.to_csv("customer_products.csv", index=False)

print("✅ customer_products.csv created with assigned owned products.")


✅ customer_products.csv created with assigned owned products.


In [80]:
import pandas as pd
import sqlite3

# Load CSVs
df_customers = pd.read_csv("customers.csv")
df_transactions = pd.read_csv("transactions.csv")
df_products = pd.read_csv("products.csv")
df_feature_store = pd.read_csv("feature_store_enhanced.csv")  # From earlier step
df_customer_products = pd.read_csv("customer_products.csv")  # From earlier step

# Create SQLite DB
conn = sqlite3.connect("cross_selling.db")

# Write tables
df_customers.to_sql("customers", conn, if_exists="replace", index=False)
df_transactions.to_sql("transactions", conn, if_exists="replace", index=False)
df_products.to_sql("products", conn, if_exists="replace", index=False)
df_feature_store.to_sql("feature_store", conn, if_exists="replace", index=False)
df_customer_products.to_sql("customer_products", conn, if_exists="replace", index=False)

conn.close()
print("✅ Data loaded into cross_selling.db")


✅ Data loaded into cross_selling.db


## LLM

In [ ]:
import sqlite3
import pandas as pd
from langchain.agents import tool

conn = sqlite3.connect("cross_selling.db")

@tool
def fetch_customer_profile(name: str) -> str:
    """Fetch basic customer profile by full name."""
    df = pd.read_sql(f"SELECT * FROM customers WHERE First_Name || ' ' || Last_Name = '{name}'", conn)
    return df.to_json(orient="records") if not df.empty else "Customer not found."

@tool
def analyze_customer_behavior(customer_id: str) -> str:
    """Provides a detailed analysis of customer behavior, spending patterns, and financial signals."""
    df = pd.read_sql(f"SELECT * FROM feature_store WHERE Customer_ID = '{customer_id}'", conn)
    if df.empty:
        return "No behavior data found for this customer."
    
    row = df.iloc[0]
    insights = []

    # 1️⃣ Aggregation Period
    insights.append(f"Analysis based on {row['Aggregation_Days']} days of transaction data.")

    # 2️⃣ Spending Overview
    insights.append(f"Total spending during this period is ${row['Total_Spend']:.2f} across {row['Num_Transactions']} transactions.")
    insights.append(f"Average transaction amount is ${row['Avg_Txn_Amount']:.2f}, with a maximum single transaction of ${row['Max_Txn_Amount']:.2f}.")
    
    # 3️⃣ Key Spending Categories
    category_flags = []
    if row.get("Spend_Grocery", 0) > 500:
        category_flags.append(f"Grocery: ${row['Spend_Grocery']:.2f}")
    if row.get("Spend_Travel", 0) > 800:
        category_flags.append(f"Travel: ${row['Spend_Travel']:.2f}")
    if row.get("Spend_Fuel", 0) > 150:
        category_flags.append(f"Fuel: ${row['Spend_Fuel']:.2f}")
    if row.get("Spend_Medical", 0) > 200:
        category_flags.append(f"Medical: ${row['Spend_Medical']:.2f}")
    if row.get("Spend_Entertainment", 0) > 300:
        category_flags.append(f"Entertainment: ${row['Spend_Entertainment']:.2f}")

    if category_flags:
        insights.append("Significant spending detected in categories: " + "; ".join(category_flags))
    else:
        insights.append(f"Primary spending category is {row['Top_Spend_Category']}.")

    # 4️⃣ Income & Salary Patterns
    if row["Has_Salary_Credit"]:
        insights.append("Regular salary deposits detected, indicating stable income.")
    if row["Salary_to_Spend_Ratio"] > 0.5:
        insights.append(f"Healthy disposable income, with a Salary-to-Spend Ratio of {row['Salary_to_Spend_Ratio']:.2f}.")

    # 5️⃣ Financial Profile
    if row["Annual_Income"] > 100000:
        insights.append(f"High annual income: ${row['Annual_Income']}.")
    elif row["Annual_Income"] > 60000:
        insights.append(f"Moderate annual income: ${row['Annual_Income']}.")

    if row["Credit_Score"] >= 750:
        insights.append(f"Excellent credit score: {row['Credit_Score']}.")
    elif row["Credit_Score"] >= 700:
        insights.append(f"Good credit score: {row['Credit_Score']}.")
    else:
        insights.append(f"Credit score is {row['Credit_Score']}.")

    # 6️⃣ Spend Variability
    if row["Spend_Variability"] > 500:
        insights.append(f"High variability in spending, suggesting inconsistent transaction amounts.")
    else:
        insights.append(f"Consistent spending behavior with low variability.")

    # 7️⃣ Idle Balance Potential
    if row["Idle_Balance_Estimate"] > 5000:
        insights.append(f"Estimated idle balance of ${row['Idle_Balance_Estimate']:.2f}, indicating potential for savings or investment products.")

    return " | ".join(insights)

@tool
def fetch_product_catalog(dummy_input: str) -> str:
    """Returns the bank's product catalog for cross-selling."""
    df = pd.read_sql("SELECT * FROM products", conn)
    return df.to_json(orient="records")


@tool
def scientific_calculator(expression: str) -> str:
    """Performs safe scientific calculations. Provide expressions like '1250 / 28' or 'sqrt(256)'."""
    import math
    allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
    allowed_names['abs'] = abs

    try:
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"




In [97]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://az-openai-document-question-answer-service.openai.azure.com/" 
OPENAI_API_KEY = "5d24331966b648738e5003caad552df8" 
OPENAI_API_VERSION = "2023-05-15"

OPENAI_DEPLOYMENT_NAME = "az-gpt_35_model"
OPENAI_MODEL_NAME="gpt-3.5-turbo"

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "az-embedding_model" 
OPENAI_ADA_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

encoding_name = "cl100k_base"

llm = AzureChatOpenAI(
                        temperature=0.1,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
import textwrap

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = [fetch_customer_profile, analyze_customer_behavior, fetch_product_catalog, scientific_calculator, fetch_owned_products ]
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "What do you recommend for David Brown"})

output_text = response.get('output', '')

# Pretty print with wrapping at 100 characters





> Entering new AgentExecutor chain...

Invoking: `fetch_customer_profile` with `{'name': 'David Brown'}`


[{"Customer_ID":"CUST0036","First_Name":"David","Last_Name":"Brown","Gender":"M","Date_of_Birth":"1967-05-13","Age":57,"Email":"jenna36@example.net","Phone":"+1-783-879-4704x9855","Address":"19364 Cole Village Apt. 859, Heatherstad, ME 82313","City":"Gonzalezmouth","State":"New York","Postal_Code":41797,"Country":"USA","Annual_Income":69823,"Credit_Score":608}]
Invoking: `analyze_customer_behavior` with `{'customer_id': 'CUST0036'}`


Analysis based on 26 days of transaction data. | Total spending during this period is $17430.46 across 7 transactions. | Average transaction amount is $2490.07, with a maximum single transaction of $7180.73. | Significant spending detected in categories: Travel: $3895.81; Fuel: $1690.41 | Regular salary deposits detected, indicating stable income. | Moderate annual income: $69823. | Credit score is 608. | High variability in spending, suggesting in

NameError: name 'DB_PATH' is not defined

In [ ]:
def format_recommendation_summary(raw_text):
    # Remove initial label if present
    raw_text = raw_text.replace("**Recommendation Summary:**", "").strip()

    # Split by numbering (assuming '1.' and '2.' structure)
    parts = raw_text.split(" 2. ")
    rec_1 = parts[0].strip()
    rec_2 = parts[1].strip() if len(parts) > 1 else ""

    formatted_output = "Recommendation Summary:\n\n"

    for idx, rec in enumerate([rec_1, rec_2], start=1):
        if not rec:
            continue
        # Split the recommendation into lines by bullet points
        lines = rec.split(" - ")
        title = lines[0].strip()
        bullets = lines[1:]

        formatted_output += f"{idx}. {title}\n"
        for bullet in bullets:
            bullet = bullet.replace("Eligibility Criteria", "Eligibility Criteria") \
                           .replace("Reason", "Reason") \
                           .replace("Benefit", "Benefit")
            formatted_output += f"   - {bullet.strip()}\n"
        formatted_output += "\n"

    return textwrap.dedent(formatted_output)

In [76]:
print(format_recommendation_summary(output_text))

Recommendation Summary:

1. 1. **Smart Shopper Card**
   - Reason: David has a high fuel spend of $1690.41 in 26 days, aligning with the 2% cashback benefit on fuel purchases.
   - Benefit: $50 cashback on the first $500 spend.
   - Eligibility Criteria: Requires an income of over $30,000 and a credit score above 650, which David meets with an annual income of $69,823 and a credit score of 608.

2. **Everyday Saver**
   - Reason: David has a potential idle balance of $52,392.54, indicating a savings opportunity. The Everyday Saver offers 4% interest and no maintenance fees.
   - Benefit: $50 bonus for maintaining a $1,000 balance for 3 months.
   - Eligibility Criteria: Open to all with a minimum $100 deposit, making it suitable for David's financial behavior and profile.

These recommendations are tailored to David's spending patterns, income level, and credit score, providing both short-term cashback benefits and long-term savings growth opportunities.


